---

## 🎉 Training Completato!

### 📋 Checklist Finale:

- ✅ Dataset verificato (features + manifests)
- ✅ fairseq2 installato correttamente
- ✅ Quick test eseguito (5 epochs)
- ✅ Training completo (50 epochs)
- ✅ BLEU score calcolato su validation set
- ✅ Checkpoint salvati e backuppati
- ✅ Risultati visualizzati e analizzati

### 🎯 Prossimi Passi:

1. **Test su Test Set**: Esegui inferenza su `manifests/test.tsv`
2. **Confronto Modelli**: Compara con baseline e altri approcci
3. **Error Analysis**: Analizza predizioni errate per miglioramenti
4. **Deployment**: Prepara modello per utilizzo in produzione

### 📊 BLEU Score Atteso:

- **Target**: 30-40% BLEU
- **Baseline**: ~0% (approcci naive)
- **SOTR paper**: ~35% BLEU su How2Sign

---

**Fine del Notebook** 🎓


In [ ]:
import shutil
from pathlib import Path
from datetime import datetime

# Crea backup directory con timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
backup_dir = Path(f"backups/sonar_full_{timestamp}")
backup_dir.mkdir(parents=True, exist_ok=True)

print("=" * 70)
print("💾 BACKUP CHECKPOINTS")
print("=" * 70)

# Files da backuppare
files_to_backup = [
    "checkpoints/sonar_full_finetuned/best_model.pt",
    "checkpoints/sonar_full_finetuned/training_log.json",
    "checkpoints/sonar_full_finetuned/training_curves.png",
    "results_validation.json",
    "results_validation.csv",
]

print(f"\n📂 Backup directory: {backup_dir}\n")

for file_path in files_to_backup:
    src = Path(file_path)
    if src.exists():
        dst = backup_dir / src.name
        shutil.copy2(src, dst)
        size_mb = dst.stat().st_size / (1024 * 1024)
        print(f"✅ {src.name:<30} ({size_mb:>6.2f} MB)")
    else:
        print(f"⚠️  {src.name:<30} (NOT FOUND)")

print(f"\n💾 Backup completato in: {backup_dir}")
print("=" * 70)

---

## 9️⃣ Backup Checkpoints su Drive

Assicurati di salvare i checkpoint migliori!


In [ ]:
import json
import pandas as pd
from pathlib import Path

# Carica risultati inferenza
results_path = Path("results_validation.json")

if results_path.exists():
    with open(results_path, "r") as f:
        results = json.load(f)

    print("=" * 70)
    print("📊 RISULTATI VALIDATION SET")
    print("=" * 70)

    print(f"\n🎯 Metriche Globali:")
    print(f"   BLEU Score: {results.get('bleu', 'N/A'):.2f}%")
    print(f"   Samples: {results.get('n_samples', 'N/A')}")

    # Mostra sample predictions
    if "predictions" in results:
        print(f"\n📝 Sample Predictions (primi 10):")
        print("─" * 70)

        df = pd.DataFrame(results["predictions"][:10])

        for idx, row in df.iterrows():
            print(f"\n{idx+1}. Video: {row['video_id']}")
            print(f"   Reference:  {row['reference']}")
            print(f"   Prediction: {row['prediction']}")
            print(f"   {'─' * 66}")

        # Salva tutti i risultati in CSV
        df_full = pd.DataFrame(results["predictions"])
        csv_path = "results_validation.csv"
        df_full.to_csv(csv_path, index=False)
        print(f"\n💾 Tutti i risultati salvati in: {csv_path}")

    print("\n" + "=" * 70)
else:
    print(f"❌ File risultati non trovato: {results_path}")

### 📊 Analisi Predizioni


In [ ]:
# Esegui inferenza su validation set completo
!python run_inference.py \
    --checkpoint checkpoints/sonar_full_finetuned/best_model.pt \
    --features_dir features/val \
    --manifest manifests/val.tsv \
    --output results_validation.json \
    --batch_size 32 \
    --device cuda

## 8️⃣ Test Inference su Validation Set


In [ ]:
import json
import torch
import matplotlib.pyplot as plt
from pathlib import Path

# Carica best model
checkpoint_path = Path("checkpoints/sonar_full_finetuned/best_model.pt")
log_path = Path("checkpoints/sonar_full_finetuned/training_log.json")

print("=" * 70)
print("📊 RISULTATI FINALI")
print("=" * 70)

if checkpoint_path.exists():
    ckpt = torch.load(checkpoint_path, map_location="cpu")

    print(f"\n🏆 BEST MODEL:")
    print(f"   Checkpoint: {checkpoint_path}")
    print(f"   Epoch: {ckpt.get('epoch', 'N/A')}")
    print(f"   BLEU Score: {ckpt.get('val_bleu', 0):.2f}%")
    print(
        f"   Train Loss: {ckpt.get('train_loss', 'N/A'):.4f}"
        if "train_loss" in ckpt
        else ""
    )

    # Plot training curves
    if log_path.exists():
        with open(log_path, "r") as f:
            log = json.load(f)

        epochs = [e["epoch"] for e in log]
        bleus = [e.get("val_bleu", 0) for e in log]
        losses = [e.get("train_loss", 0) for e in log]

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

        # BLEU curve
        ax1.plot(epochs, bleus, "b-o", linewidth=2, markersize=6)
        ax1.set_xlabel("Epoch", fontsize=12)
        ax1.set_ylabel("BLEU Score (%)", fontsize=12)
        ax1.set_title("Validation BLEU Score", fontsize=14, fontweight="bold")
        ax1.grid(True, alpha=0.3)
        ax1.axhline(
            y=max(bleus),
            color="r",
            linestyle="--",
            alpha=0.5,
            label=f"Best: {max(bleus):.2f}%",
        )
        ax1.legend()

        # Loss curve
        ax2.plot(epochs, losses, "r-o", linewidth=2, markersize=6)
        ax2.set_xlabel("Epoch", fontsize=12)
        ax2.set_ylabel("Loss", fontsize=12)
        ax2.set_title("Training Loss", fontsize=14, fontweight="bold")
        ax2.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.savefig(
            "checkpoints/sonar_full_finetuned/training_curves.png",
            dpi=150,
            bbox_inches="tight",
        )
        plt.show()

        print(f"\n📈 Training curves salvate in: training_curves.png")

    # Statistiche training
    print(f"\n📊 STATISTICHE:")
    print(f"   Total epochs: {len(log) if log_path.exists() else 'N/A'}")
    print(f"   Best BLEU: {max(bleus):.2f}%" if log_path.exists() else "")
    print(f"   Final Loss: {losses[-1]:.4f}" if log_path.exists() and losses else "")

else:
    print("\n❌ Checkpoint non trovato!")
    print(f"   Path: {checkpoint_path}")

print("\n" + "=" * 70)

---

## 7️⃣ Analisi Risultati Finali


In [ ]:
import time
import json
from pathlib import Path
from IPython.display import clear_output

log_file = Path("checkpoints/sonar_full_finetuned/training_log.json")

if not log_file.exists():
    print(f"⏳ Waiting for training to start...")
    print(f"   Log file: {log_file}")
else:
    # Monitor ogni 30 secondi
    for _ in range(100):  # Max 50 minuti di monitoring
        clear_output(wait=True)

        try:
            with open(log_file, "r") as f:
                log = json.load(f)

            print("=" * 70)
            print("📊 TRAINING PROGRESS")
            print("=" * 70)

            # Mostra ultimi 10 epoch
            recent = log[-10:] if len(log) > 10 else log

            for entry in recent:
                epoch = entry.get("epoch", "?")
                train_loss = entry.get("train_loss", 0)
                val_bleu = entry.get("val_bleu", 0)
                print(
                    f"Epoch {epoch:>3}: Loss={train_loss:>7.4f} | BLEU={val_bleu:>5.2f}%"
                )

            # Best model so far
            best_entry = max(log, key=lambda x: x.get("val_bleu", 0))
            print("\n" + "─" * 70)
            print(
                f"🏆 BEST: Epoch {best_entry['epoch']} | BLEU = {best_entry['val_bleu']:.2f}%"
            )
            print("=" * 70)

        except Exception as e:
            print(f"⚠️ Error reading log: {e}")

        time.sleep(30)  # Aggiorna ogni 30 secondi

## 6️⃣ Monitoring Training (Esegui in Parallelo)

Mentre il training è in corso, puoi monitorare i progressi con questa cella.


In [ ]:
%%time
# Training completo - Full dataset, 50 epochs

# IMPORTANTE: Assicurati di essere nella directory corretta!
import os
os.chdir('/content/drive/MyDrive/How2Sign_SONAR')
print(f"📂 Working directory: {os.getcwd()}\n")

!python train_sonar_finetuning.py \
    --encoder_checkpoint checkpoints/sonar_encoder_finetuned/best_encoder.pt \
    --train_features features/train \
    --train_manifest manifests/train.tsv \
    --val_features features/val \
    --val_manifest manifests/val.tsv \
    --output_dir checkpoints/sonar_full_finetuned \
    --epochs 50 \
    --batch_size 32 \
    --learning_rate 1e-4 \
    --eval_every 5 \
    --save_every 10 \
    --device cuda

print("\n" + "=" * 70)
print("✅ TRAINING COMPLETATO!")
print("=" * 70)

---

## 5️⃣ Training Completo (50 epochs, full dataset)

⚠️ **ESEGUI SOLO SE IL QUICK TEST È ANDATO BENE!**

⏱️ Tempo stimato: **2-4 ore** su GPU T4


In [ ]:
import json
from pathlib import Path

# Verifica checkpoint salvato
test_ckpt = Path("checkpoints/sonar_full_test/best_model.pt")
if test_ckpt.exists():
    print("✅ Quick test completato con successo!")
    print(f"   Checkpoint: {test_ckpt}")

    # Carica info checkpoint
    import torch

    ckpt = torch.load(test_ckpt, map_location="cpu")
    print(f"\n📊 Risultati:")
    print(f"   Best Epoch: {ckpt.get('epoch', 'N/A')}")
    print(f"   Best BLEU: {ckpt.get('val_bleu', 0):.2f}%")
    print(
        f"   Train Loss: {ckpt.get('train_loss', 'N/A'):.4f}"
        if "train_loss" in ckpt
        else ""
    )

    print("\n🚀 Procedi con il training completo nella prossima cella!")
else:
    print("❌ Quick test fallito - verifica errori sopra")

### 📊 Verifica Risultati Quick Test


In [ ]:
%%time
# Quick test - 5 epochs, 100 samples per verificare setup

# IMPORTANTE: Assicurati di essere nella directory corretta!
import os
os.chdir('/content/drive/MyDrive/How2Sign_SONAR')
print(f"📂 Working directory: {os.getcwd()}")
print(f"📄 Script exists: {os.path.exists('train_sonar_finetuning.py')}\n")

!python train_sonar_finetuning.py \
    --encoder_checkpoint checkpoints/sonar_encoder_finetuned/best_encoder.pt \
    --train_features features/train \
    --train_manifest manifests/train.tsv \
    --val_features features/val \
    --val_manifest manifests/val.tsv \
    --output_dir checkpoints/sonar_full_test \
    --epochs 5 \
    --batch_size 16 \
    --learning_rate 1e-4 \
    --max_samples 100 \
    --device cuda

---

## 4️⃣ Quick Test (5 epochs, 100 samples)

Prima di lanciare il training completo, facciamo un test veloce per verificare che tutto funzioni.

**✅ LO SCRIPT È STATO AGGIORNATO!**

Modifiche principali:
- ✅ Usa API SONAR corretta (`sonar-space` package)
- ✅ Import da `sonar.inference_pipelines.text` invece di `fairseq2.models.sonar`
- ✅ Decoder SONAR reale (non più placeholder)
- ✅ Target embeddings calcolati da testi veri
- ✅ Gradient clipping per stabilità

Vedi dettagli completi in: `SONAR_SCRIPT_FIX.md`

---

**⚠️ PREREQUISITO**: Assicurati di aver eseguito:

1. ✅ Cella 1 (Mount Drive)
2. ✅ Cella 2 (Verifica Dataset)
3. ✅ Cella 3-bis (Verifica Setup) → SONAR deve essere installato!

---


### ✅ Verifica Pre-Training

**Esegui questa cella PRIMA del Quick Test** per verificare che tutto sia pronto.


In [ ]:
import os
from pathlib import Path

print("=" * 70)
print("✅ CHECKLIST PRE-TRAINING")
print("=" * 70)

checks_passed = 0
total_checks = 5

# 1. Verifica directory corretta
print("\n1️⃣ Verifica directory di lavoro...")
expected_dir = "/content/drive/MyDrive/How2Sign_SONAR"
current_dir = os.getcwd()

if current_dir == expected_dir:
    print(f"   ✅ Directory corretta: {current_dir}")
    checks_passed += 1
else:
    print(f"   ❌ Directory errata!")
    print(f"      Attuale: {current_dir}")
    print(f"      Attesa:  {expected_dir}")
    print(f"   🔧 Esegui: os.chdir('{expected_dir}')")

# 2. Verifica script esiste
print("\n2️⃣ Verifica script train_sonar_finetuning.py...")
script_path = Path("train_sonar_finetuning.py")
if script_path.exists():
    print(f"   ✅ Script trovato: {script_path}")
    checks_passed += 1
else:
    print(f"   ❌ Script NON trovato!")
    print(f"   🔧 Assicurati di aver caricato lo script su Drive")

# 3. Verifica SONAR installato
print("\n3️⃣ Verifica SONAR installato...")
try:
    from sonar.inference_pipelines.text import TextToEmbeddingModelPipeline

    print(f"   ✅ SONAR importato correttamente")
    checks_passed += 1
except ImportError:
    print(f"   ❌ SONAR non installato!")
    print(f"   🔧 Torna alle celle di installazione")

# 4. Verifica encoder checkpoint
print("\n4️⃣ Verifica encoder checkpoint...")
encoder_path = Path("checkpoints/sonar_encoder_finetuned/best_encoder.pt")
if encoder_path.exists():
    print(f"   ✅ Encoder checkpoint trovato")
    checks_passed += 1
else:
    print(f"   ❌ Encoder checkpoint NON trovato!")
    print(f"   🔧 Path: {encoder_path}")

# 5. Verifica features e manifests
print("\n5️⃣ Verifica dataset...")
train_manifest = Path("manifests/train.tsv")
val_manifest = Path("manifests/val.tsv")
train_features = Path("features/train")
val_features = Path("features/val")

if all(
    [
        train_manifest.exists(),
        val_manifest.exists(),
        train_features.exists(),
        val_features.exists(),
    ]
):
    print(f"   ✅ Dataset completo (features + manifests)")
    checks_passed += 1
else:
    print(f"   ❌ Dataset incompleto!")
    print(f"      train.tsv: {train_manifest.exists()}")
    print(f"      val.tsv: {val_manifest.exists()}")
    print(f"      features/train: {train_features.exists()}")
    print(f"      features/val: {val_features.exists()}")

# Risultato finale
print("\n" + "=" * 70)
print(f"📊 RISULTATO: {checks_passed}/{total_checks} controlli superati")
print("=" * 70)

if checks_passed == total_checks:
    print("\n🎉 TUTTO PRONTO!")
    print("👉 Procedi con il Quick Test nella cella successiva")
else:
    print(f"\n⚠️  {total_checks - checks_passed} problemi da risolvere")
    print("👉 Risolvi i problemi sopra prima di continuare")

print("=" * 70)

In [ ]:
# ============================================================================
# INSTALLA DIPENDENZE AGGIUNTIVE
# ============================================================================
# NOTA: Questo presuppone che tu abbia GIÀ eseguito setup iniziale:
#   1. pip install numpy>=2,<2.3
#   2. pip install torch 2.9.0 + cu126
#   3. pip install fairseq2 --no-deps
# ============================================================================

print("=" * 70)
print("📦 INSTALLAZIONE DIPENDENZE AGGIUNTIVE")
print("=" * 70)

# Installa dipendenze mancanti di fairseq2 (mantenendo numpy 2.x)
print("\n1️⃣ Installazione componenti fairseq2...")
!pip install fairseq2n==0.7.0

# INSTALLA SONAR (pacchetto SEPARATO da fairseq2!)
print("\n2️⃣ Installazione SONAR (sonar-space)...")
!pip install "sonar-space>=0.5.0"

# Dipendenze aggiuntive
print("\n3️⃣ Installazione dipendenze extra...")
!pip install sentencepiece pyyaml tqdm sacrebleu pandas

# Verifica installazione SONAR
print("\n4️⃣ Verifica installazione SONAR...")
!pip show sonar-space

# RIAVVIO RUNTIME NECESSARIO per caricare nuovi moduli
print("\n" + "=" * 70)
print("⚠️  IMPORTANTE: RIAVVIA IL RUNTIME ORA!")
print("=" * 70)
print("\n📝 Dopo il riavvio:")
print("   1. Riesegui dalla Cella 1 (Mount Drive)")
print("   2. Salta la Cella 3 (già eseguita)")
print("   3. Continua con Cella 3-bis (Verifica)")
print("\n⚠️  Runtime → Restart runtime")
print("=" * 70)

## 3️⃣ Installa Dipendenze

⚠️ **PREREQUISITO**: Assicurati di aver già eseguito PRIMA del notebook:

```python
# Setup iniziale (ESEGUI PRIMA DEL NOTEBOOK)
!pip install -U "numpy>=2,<2.3"
!pip install -U torch==2.9.0 torchvision==0.24.0 torchaudio==2.9.0 --index-url https://download.pytorch.org/whl/cu126
!pip install -U --no-deps fairseq2 --extra-index-url https://fair.pkg.atmeta.com/fairseq2/whl/pt2.9.0/cu126
```

Questa cella installa:

1. **fairseq2n** - Componente nativo con modelli base
2. **sonar-space** - Pacchetto SONAR (SEPARATO da fairseq2!)
3. **Dipendenze extra** - sentencepiece, pyyaml, sacrebleu, etc.

**IMPORTANTE**: SONAR NON è parte di fairseq2, è un progetto separato!

Lo script `train_sonar_finetuning.py` è stato **RISCRITTO** per usare l'API corretta.


### 🔍 Diagnostica Pre-Installazione

Esegui questa cella PRIMA della cella 3 per vedere cosa è già installato.


In [ ]:
import subprocess
import sys

print("=" * 70)
print("🔍 DIAGNOSTICA PACCHETTI INSTALLATI")
print("=" * 70)

packages_to_check = [
    ("numpy", "NumPy"),
    ("torch", "PyTorch"),
    ("fairseq2", "fairseq2"),
    ("fairseq2n", "fairseq2n"),
    ("sonar-space", "SONAR"),
    ("sentencepiece", "SentencePiece"),
    ("sacrebleu", "SacreBLEU"),
]

print("\n📦 Pacchetti installati:\n")

for package, name in packages_to_check:
    result = subprocess.run(
        [sys.executable, "-m", "pip", "show", package], capture_output=True, text=True
    )

    if result.returncode == 0:
        # Estrai versione
        for line in result.stdout.split("\n"):
            if line.startswith("Version:"):
                version = line.split("Version:")[1].strip()
                print(f"✅ {name:<20} v{version}")
                break
    else:
        print(f"❌ {name:<20} NON INSTALLATO")

print("\n" + "=" * 70)
print("📝 PROSSIMI PASSI:")
print("=" * 70)

# Controlla se sonar-space è installato
sonar_result = subprocess.run(
    [sys.executable, "-m", "pip", "show", "sonar-space"], capture_output=True, text=True
)

if sonar_result.returncode != 0:
    print("\n⚠️  sonar-space NON è installato!")
    print("   → Procedi con Cella 3 per installarlo")
else:
    print("\n✅ sonar-space è già installato!")
    print("   → Salta Cella 3 e vai a Cella 3-bis (Verifica)")

print("=" * 70)

In [ ]:
print("=" * 70)
print("✅ VERIFICA SETUP COMPLETO")
print("=" * 70)

# Verifica PyTorch
import torch

print(f"\n🔥 PyTorch: {torch.__version__}")
print(f"   CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   CUDA version: {torch.version.cuda}")
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
else:
    print(f"   ⚠️ CUDA not available - training su CPU (molto lento!)")

# Verifica numpy
import numpy as np

print(f"\n📊 NumPy: {np.__version__}")
if not np.__version__.startswith("2."):
    print(f"   ❌ ERROR: NumPy dovrebbe essere 2.x, trovato {np.__version__}")
    print(f"   🔧 SOLUZIONE: Esegui setup iniziale di nuovo")
else:
    print(f"   ✅ NumPy versione corretta")

# Verifica fairseq2
print(f"\n🔍 Verifica fairseq2...")
try:
    import fairseq2

    print(f"   ✅ fairseq2: v{fairseq2.__version__}")

    # Verifica fairseq2n
    import fairseq2n

    print(f"   ✅ fairseq2n disponibile")

except ModuleNotFoundError as e:
    print(f"   ❌ ModuleNotFoundError: {e}")
    print(f"\n🔧 PROBLEMA: Modulo mancante")
    print(f"   SOLUZIONE:")
    print(f"   1. Torna alla Cella 3")
    print(f"   2. Riesegui l'installazione")
    print(f"   3. Riavvia runtime")

except Exception as e:
    print(f"   ❌ ERROR: {e}")

# Verifica SONAR (pacchetto SEPARATO!)
print(f"\n🎯 Verifica SONAR (sonar-space)...")
try:
    import sonar

    print(f"   ✅ sonar: disponibile")

    # Test import pipelines SONAR
    from sonar.inference_pipelines.text import (
        TextToEmbeddingModelPipeline,
        EmbeddingToTextModelPipeline,
    )

    print(f"   ✅ SONAR text pipelines importate!")

    print(f"\n" + "=" * 70)
    print(f"🎉 TUTTO PRONTO PER IL TRAINING!")
    print(f"=" * 70)
    print(f"\n👉 Procedi con Cella 4 (Quick Test)")

except ModuleNotFoundError as e:
    print(f"   ❌ ModuleNotFoundError: {e}")
    print(f"\n🔧 PROBLEMA: SONAR non installato")
    print(f"   SOLUZIONE:")
    print(f"   1. Torna alla Cella 3")
    print(f"   2. Verifica che sonar-space sia installato")
    print(f"   3. Riavvia runtime")
    print(f"   4. Riesegui Celle 1, 2, poi questa")

except Exception as e:
    print(f"   ❌ ERROR: {e}")
    print(f"   Tipo: {type(e).__name__}")

print("\n" + "=" * 70)

In [ ]:
# Test identico a quello che DOVREBBE fare lo script
print("🧪 Test importazione SONAR (pacchetto corretto)...")
print("=" * 70)

# Prima verifica se il pacchetto è installato
import subprocess
import sys

print("\n📦 Verifica pacchetto sonar-space installato...")
result = subprocess.run(
    [sys.executable, "-m", "pip", "show", "sonar-space"], capture_output=True, text=True
)

if result.returncode == 0:
    print("✅ sonar-space è installato!")
    print(f"\n{result.stdout}")
else:
    print("❌ sonar-space NON è installato!")
    print("\n🔧 INSTALLA ORA:")
    print('   !pip install "sonar-space>=0.5.0"')
    print("\nPoi riavvia il runtime e riprova.")

print("\n" + "=" * 70)
print("🧪 Test import SONAR pipelines...")
print("=" * 70)

SONAR_AVAILABLE = False

try:
    # SONAR è un pacchetto SEPARATO da fairseq2!
    from sonar.inference_pipelines.text import (
        TextToEmbeddingModelPipeline,
        EmbeddingToTextModelPipeline,
    )

    SONAR_AVAILABLE = True
    print("✅ SUCCESS: SONAR text pipelines importati!")
    print(f"   SONAR_AVAILABLE = {SONAR_AVAILABLE}")
    print("\n🎉 SONAR funziona correttamente!")
    print("\n✅ Lo script train_sonar_finetuning.py è stato AGGIORNATO!")
    print("   - Usa: from sonar.inference_pipelines.text import ...")
    print("   - Dovrebbe funzionare correttamente ora!")

except ImportError as e:
    print(f"❌ ImportError: {e}")
    print(f"   SONAR_AVAILABLE = {SONAR_AVAILABLE}")
    print("\n🔴 PROBLEMA: SONAR non importabile!")
    print("\n💡 SOLUZIONE:")
    print("   1. Torna alla Cella 3 e riesegui (senza -q)")
    print("   2. Verifica che l'installazione completi senza errori")
    print("   3. Runtime → Restart runtime")
    print("   4. Riesegui Celle 1, 2, poi questa")
    print("\n⚠️  Se persiste, prova:")
    print("   !pip uninstall -y sonar-space")
    print('   !pip install "sonar-space==0.5.0" --no-cache-dir')

except Exception as e:
    print(f"❌ Errore: {e}")
    print(f"   Tipo: {type(e).__name__}")
    print(f"   SONAR_AVAILABLE = {SONAR_AVAILABLE}")

print("=" * 70)

### 🔧 Installazione Manuale SONAR

**ESEGUI QUESTA CELLA** se il test sopra ha mostrato che `sonar-space` non è installato.


In [ ]:
print("=" * 70)
print("🔧 INSTALLAZIONE MANUALE SONAR-SPACE")
print("=" * 70)

# Installazione con output completo (no -q)
print("\n📦 Installazione sonar-space con dipendenze...")
!pip install "sonar-space>=0.5.0" --no-cache-dir

print("\n" + "=" * 70)
print("✅ INSTALLAZIONE COMPLETATA!")
print("=" * 70)

# Verifica immediata
import subprocess
import sys

result = subprocess.run(
    [sys.executable, "-m", "pip", "show", "sonar-space"],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("\n✅ sonar-space è stato installato correttamente!")
    print(f"\n{result.stdout}")
    print("\n" + "=" * 70)
    print("⚠️  RIAVVIA IL RUNTIME ORA!")
    print("=" * 70)
    print("\n📝 Dopo il riavvio:")
    print("   1. Riesegui Celle 1, 2")
    print("   2. Esegui cella 'Test Importazione SONAR'")
    print("   3. Se funziona ✅ → Procedi con Quick Test")
else:
    print("\n❌ Installazione fallita!")
    print("\n🔧 ERRORI DA VERIFICARE:")
    print("   - Compatibilità versioni PyTorch/fairseq2?")
    print("   - Dipendenze mancanti?")
    print("\n💡 Prova versione specifica:")
    print('   !pip install "sonar-space==0.5.0" --no-cache-dir')

print("=" * 70)

### 🔧 Alternativa: Installazione con Versione Specifica

**Esegui SOLO se l'installazione sopra ha dato errori**. Usa versione esatta invece di `>=0.5.0`.


In [ ]:
print("=" * 70)
print("🔧 INSTALLAZIONE SONAR-SPACE (Versione Esatta)")
print("=" * 70)

# Disinstalla prima se esiste
print("\n1️⃣ Rimozione versione precedente (se esiste)...")
!pip uninstall -y sonar-space

# Installa versione specifica 0.5.0
print("\n2️⃣ Installazione sonar-space==0.5.0...")
!pip install "sonar-space==0.5.0" --no-cache-dir -v

# Verifica
print("\n3️⃣ Verifica installazione...")
!pip show sonar-space

print("\n" + "=" * 70)
print("⚠️  RIAVVIA IL RUNTIME!")
print("=" * 70)
print("\nDopo riavvio: Celle 1, 2 → Test Importazione")
print("=" * 70)

---

### ✅ AGGIORNAMENTO: train_sonar_finetuning.py è stato CORRETTO!

**Problema (RISOLTO)**: Lo script cercava di importare:
```python
from fairseq2.models.sonar import load_sonar_text_decoder  # ❌ NON ESISTE
```

**Soluzione (APPLICATA)**: Lo script ora usa:
```python
from sonar.inference_pipelines.text import (
    TextToEmbeddingModelPipeline,
    EmbeddingToTextModelPipeline,
)
```

**Dettagli**:
- **Nome pacchetto**: `sonar-space` (su PyPI)
- **Import corretto**: `from sonar.inference_pipelines.text import ...`
- **Documentazione**: https://github.com/facebookresearch/SONAR

**✅ Lo script è stato riscritto e dovrebbe funzionare dopo l'installazione di sonar-space!**

Vedi dettagli completi in: `SONAR_SCRIPT_FIX.md`

---


### 🧪 Test Importazione SONAR (pacchetto corretto)

**PROBLEMA RISOLTO**: `fairseq2.models.sonar` NON ESISTE!

SONAR è un **pacchetto separato** chiamato `sonar-space`, non parte di fairseq2.


## 3️⃣ bis - Verifica Setup (DOPO RIAVVIO)

✅ **Esegui questa cella DOPO aver riavviato il runtime dalla cella precedente**


In [ ]:
import os
from pathlib import Path

print("=" * 70)
print("📊 VERIFICA DATASET")
print("=" * 70)

# 1. Conta features (supporta .pt e .npy)
train_features = len(list(Path("features/train").glob("*.pt"))) + len(
    list(Path("features/train").glob("*.npy"))
)
val_features = len(list(Path("features/val").glob("*.pt"))) + len(
    list(Path("features/val").glob("*.npy"))
)
test_features = len(list(Path("features/test").glob("*.pt"))) + len(
    list(Path("features/test").glob("*.npy"))
)

print("\n📦 FEATURES DISPONIBILI:")
print(f"   Train: {train_features:>4} files")
print(f"   Val:   {val_features:>4} files")
print(f"   Test:  {test_features:>4} files")
print(f"   {'─' * 20}")
print(f"   TOTAL: {train_features + val_features + test_features:>4} files")

# 2. Verifica manifests
print("\n📋 MANIFESTS:")
for split in ["train", "val", "test"]:
    manifest_path = Path(f"manifests/{split}.tsv")
    if manifest_path.exists():
        n_lines = sum(1 for _ in open(manifest_path)) - 1  # -1 header
        print(f"   ✅ {split}.tsv: {n_lines} samples")
    else:
        print(f"   ❌ {split}.tsv: NOT FOUND")

# 3. Verifica sample feature
sample_files = list(Path("features/train").glob("*.pt")) + list(
    Path("features/train").glob("*.npy")
)
if sample_files:
    sample_file = sample_files[0]
    print(f"\n📐 SAMPLE FEATURE:")

    if sample_file.suffix == ".pt":
        import torch

        data = torch.load(sample_file, map_location="cpu")
        features = data["features"]
        print(f"   Shape: {features.shape}")
        print(f"   Format: PyTorch (.pt)")
        print(f"   Video ID: {data.get('video_id', 'N/A')}")
        print(f"   Text: {data.get('text', 'N/A')[:60]}...")
    else:
        import numpy as np

        features = np.load(sample_file)
        print(f"   Shape: {features.shape}")
        print(f"   Format: NumPy (.npy)")
        print(f"   File: {sample_file.name}")
else:
    print("\n❌ Nessuna feature trovata!")

# 4. Verifica encoder checkpoint
encoder_ckpt = Path("checkpoints/sonar_encoder_finetuned/best_encoder.pt")
print(f"\n🔍 ENCODER CHECKPOINT:")
if encoder_ckpt.exists():
    import torch

    ckpt = torch.load(encoder_ckpt, map_location="cpu")
    print(f"   ✅ Found: {encoder_ckpt}")
    print(f"   Epoch: {ckpt.get('epoch', 'N/A')}")
    print(
        f"   Val Loss: {ckpt.get('val_loss', 'N/A'):.4f}"
        if "val_loss" in ckpt
        else "   Val Loss: N/A"
    )
else:
    print(f"   ⚠️ NOT FOUND - training partirà da zero")

print("\n" + "=" * 70)

## 2️⃣ Verifica Dataset e Features


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Cambia directory al progetto
import os
os.chdir('/content/drive/MyDrive/How2Sign_SONAR')

print("✅ Google Drive montato")
print("\n📂 Struttura directory:")
!ls -lh

## 1️⃣ Mount Google Drive e Setup


# 🎓 SONAR Encoder-Decoder Fine-Tuning su How2Sign

Questo notebook esegue il fine-tuning completo di **SONAR Encoder + SONAR Decoder** per traduzione ASL→English.

## 📋 Pipeline:

1. **SONAR Encoder** (fine-tuned su ASL) → Embedding 1024-dim
2. **SONAR Decoder** (pre-trained multilingue) → Testo inglese

## 🎯 Obiettivo:

- BLEU Score atteso: **30-40%** (vs 0% con approcci errati)
- Dataset: How2Sign (train, val, test)

---

**Autore**: Ignazio Picciche  
**Data**: Novembre 2024
